In [1]:
import pandas as pd
import re
import os
import collections
import nltk
import urllib
import bs4 as bs
import re
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import numpy

In [2]:
list_of_files = os.listdir('../top1000_complete')

In [3]:
list_of_files2= []
for i in list_of_files:
    nome = '../top1000_complete/' + i + '/' + 'Documents_xml/' +os.listdir(f'../top1000_complete/{i}/Documents_xml')[0]
    list_of_files2.append(nome)

In [4]:
lines = []
for file in list_of_files2:
    f= open(file,'r')
    lines.append(f.readlines())
    f.close()

In [5]:
paper1= lines[2]

In [6]:
paper1

['<PAPER>\n',
 '  <S sid="0">Advances In Domain Independent Linear Text Segmentation</S>\n',
 '  <ABSTRACT>\n',
 '    <S sid="1" ssid="1">This paper describes a method for linear text segmentation which is twice as accurate and over seven times as fast as the state-of-the-art (Reynar, 1998).</S>\n',
 '    <S sid="2" ssid="2">Inter-sentence similarity is replaced by rank in the local context.</S>\n',
 '    <S sid="3" ssid="3">Boundary locations are discovered by divisive clustering.</S>\n',
 '  </ABSTRACT>\n',
 '  <SECTION title="1 Introduction" number="1">\n',
 '    <S sid="4" ssid="1">Even moderately long documents typically address several topics or different aspects of the same topic.</S>\n',
 '    <S sid="5" ssid="2">The aim of linear text segmentation is to discover the topic boundaries.</S>\n',
 '    <S sid="6" ssid="3">The uses of this procedure include information retrieval (Hearst and Plaunt, 1993; Hearst, 1994; Yaari, 1997; Reynar, 1999), summarization (Reynar, 1998), text un

lista3=[]
lines2=''
for i in range(len(paper1)):
    for j in paper1[i]:
        lines2= lines2+j
    lista3.append(lines2)

In [7]:
texto=''
for i in paper1:
    texto = texto + str(i)

In [8]:
def preprocessa_intro(corpus: str):
    regIter = re.finditer('(<SECTION.*Introduction.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = re.sub(',','', txt)
        txt = re.sub(';','', txt)
        txt = re.sub(':','', txt)
        txt = re.sub('[ ]+',' ', txt)
        txt = re.sub(r'\([^)]*\)', "",txt)
        txt = re.sub(r'\<(.*?)\>',"",txt)
        txt = re.sub(r'small\&',"",txt)
        txt = re.sub(r'tagger&amp',"",txt)
        txt = re.sub(r';quot',"",txt)
        txt = re.sub(r'\&amp',"",txt)
        txt = re.sub(' +', ' ', txt)
        txt = txt.lower()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = txt.strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        return txt
    return txt

def preprocessa_abstract(corpus: str):
    regIter = re.finditer("(<ABSTRACT>)(.+?)(</ABSTRACT>)", corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = re.sub(',','', txt)
        txt = re.sub(';',' ;', txt)
        txt = re.sub('[ ]+',' ', txt)
        txt = re.sub(r'\([^)]*\)', "",txt)
        txt = re.sub(r'\<(.*?)\>',"",txt)
        txt = txt.lower()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        lista.append(txt)
    return txt


def preprocessa_conclusion(corpus: str):
    regIter = re.finditer('(<SECTION.*Conclusion.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = re.sub(',','', txt)
        txt = re.sub(';','', txt)
        txt = re.sub(':','', txt)
        txt = re.sub('[ ]+',' ', txt)
        txt = re.sub(r'\([^)]*\)', "",txt)
        txt = re.sub(r'\<(.*?)\>',"",txt)
        txt = re.sub(r'small\&',"",txt)
        txt = re.sub(r'tagger&amp',"",txt)
        txt = re.sub(r';quot',"",txt)
        txt = re.sub(r'\&amp',"",txt)
        txt = re.sub(' +', ' ', txt)
        txt = txt.lower()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = txt.strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        return txt
    return txt

In [9]:
preprocessa_conclusion(texto)

"a segmentation algorithm has two key elements a clustering strategy and a similarity measure. our results show divisive clustering is more precise than sliding window and lexical chains for locating topic boundaries. four similarity measures were examined. the cosine coefficient and dot density measure yield similar results. our spread activation based semantic measure ) improved accuracy. this confirms that although kozima's approach is computationally expensive it does produce more precise segmentation. the most significant improvement was due to our ranking scheme which linearises the cosine coefficient. our experiments demonstrate that given insufficient data the qualitative behaviour of the cosine measure is indeed more reliable than the actual values. although our evaluation scheme is sufficient for this comparative study further research requires a large scale task independent benchmark. it would be interesting to compare c99 with the multi-source method described in using the 

In [10]:
abstract   = preprocessa_abstract(texto)
#intro      = preprocessa_intro(texto)
conclusion = preprocessa_conclusion(texto)

In [11]:
abstract

' this paper describes a method for linear text segmentation which is twice as accurate and over seven times as fast as the state-of-the-art . inter-sentence similarity is replaced by rank in the local context. boundary locations are discovered by divisive clustering. '

In [12]:
token_abstract   = preprocessa_abstract(texto).split()
token_intro      = preprocessa_intro(texto).split()
token_conclusion = preprocessa_conclusion(texto).split()

In [13]:
tokens = [token_abstract+token_intro+token_conclusion]

In [14]:
tokens[0]

['this',
 'paper',
 'describes',
 'a',
 'method',
 'for',
 'linear',
 'text',
 'segmentation',
 'which',
 'is',
 'twice',
 'as',
 'accurate',
 'and',
 'over',
 'seven',
 'times',
 'as',
 'fast',
 'as',
 'the',
 'state-of-the-art',
 '.',
 'inter-sentence',
 'similarity',
 'is',
 'replaced',
 'by',
 'rank',
 'in',
 'the',
 'local',
 'context.',
 'boundary',
 'locations',
 'are',
 'discovered',
 'by',
 'divisive',
 'clustering.',
 'even',
 'moderately',
 'long',
 'documents',
 'typically',
 'address',
 'several',
 'topics',
 'or',
 'different',
 'aspects',
 'of',
 'the',
 'same',
 'topic.',
 'the',
 'aim',
 'of',
 'linear',
 'text',
 'segmentation',
 'is',
 'to',
 'discover',
 'the',
 'topic',
 'boundaries.',
 'the',
 'uses',
 'of',
 'this',
 'procedure',
 'include',
 'information',
 'retrieval',
 'summarization',
 'text',
 'understanding',
 'anaphora',
 'resolution',
 'language',
 'modelling',
 'and',
 'improving',
 'document',
 'navigation',
 'for',
 'the',
 'visually',
 'disabled',
 '.

In [15]:
for i in range(len(tokens)):
    tokens[i] = [word for word in tokens[i] if word not in stopwords.words('English')]

In [16]:
for i in tokens:
    if 'this' in i:
        print(i)

In [17]:
tokens[0]

['paper',
 'describes',
 'method',
 'linear',
 'text',
 'segmentation',
 'twice',
 'accurate',
 'seven',
 'times',
 'fast',
 'state-of-the-art',
 '.',
 'inter-sentence',
 'similarity',
 'replaced',
 'rank',
 'local',
 'context.',
 'boundary',
 'locations',
 'discovered',
 'divisive',
 'clustering.',
 'even',
 'moderately',
 'long',
 'documents',
 'typically',
 'address',
 'several',
 'topics',
 'different',
 'aspects',
 'topic.',
 'aim',
 'linear',
 'text',
 'segmentation',
 'discover',
 'topic',
 'boundaries.',
 'uses',
 'procedure',
 'include',
 'information',
 'retrieval',
 'summarization',
 'text',
 'understanding',
 'anaphora',
 'resolution',
 'language',
 'modelling',
 'improving',
 'document',
 'navigation',
 'visually',
 'disabled',
 '.',
 'paper',
 'focuses',
 'domain',
 'independent',
 'methods',
 'segmenting',
 'written',
 'text.',
 'present',
 'new',
 'algorithm',
 'builds',
 'previous',
 'work',
 'reynar',
 '.',
 'primary',
 'distinction',
 'method',
 'use',
 'ranking',
 '

In [18]:
model = Word2Vec(tokens, min_count=1)

In [19]:
model.corpus_total_words

1524

In [20]:
model.wv['improved']

array([-7.9836529e-03, -8.0998410e-03,  1.1233829e-03, -6.9632992e-04,
       -9.4498228e-03,  2.7337989e-03,  2.2812416e-03,  5.0073252e-03,
        4.2019379e-03,  3.2462860e-03, -9.9586602e-03,  5.3387024e-03,
        4.3387804e-03, -7.7758804e-03, -1.9766539e-03,  6.5006810e-05,
       -6.3308800e-04, -1.2210065e-03, -5.6054578e-03, -4.0849642e-04,
        4.5191338e-03, -4.2409409e-04,  7.9021533e-04,  2.0240892e-03,
       -4.0527866e-03, -7.7603832e-03,  9.0039419e-03, -9.9387541e-03,
       -2.6854014e-03, -1.1871197e-03, -7.3451288e-03,  1.9306235e-03,
        8.1562465e-03, -1.6512889e-03,  7.7571627e-03,  7.1540411e-04,
       -5.6799226e-03, -2.3288834e-03, -8.6285477e-04,  5.6965374e-03,
        7.0212842e-03,  5.9895823e-03, -6.1499830e-03, -7.0703803e-03,
        6.3317763e-03,  2.3368632e-03,  5.1321052e-03,  4.8025927e-04,
        8.1045358e-03, -8.8239359e-03,  9.9537075e-03, -8.4356051e-03,
       -1.7137274e-03,  6.3094408e-03, -9.5526259e-03,  4.4656270e-03,
      

In [21]:
cosine_similarity = numpy.dot(model.wv['improved'], model.wv['segmentation'])/(numpy.linalg.norm(model.wv['improved'])* numpy.linalg.norm(model.wv['segmentation']))
cosine_similarity

-0.049045265

In [22]:
model.wv.similarity('improved', 'segmentation')

-0.04904526

In [23]:
frases_abstract = abstract.split('.')

In [24]:
palavras = frases_abstract[0].split(' ')

In [25]:
soma= 0
for i in palavras:
    try:
        soma = soma + model.wv[i]
    except:
        soma = soma + 0

In [26]:
frases_conclusion = conclusion.split('.')

In [27]:
palavras = frases_conclusion[0].split(' ')

In [28]:
soma2= 0
for i in palavras:
    try:
        soma2 = soma2 + model.wv[i]
    except:
        soma2 = soma2 + 0

In [29]:
cosine_similarity = numpy.dot(soma, soma2)/(numpy.linalg.norm(soma)* numpy.linalg.norm(soma2))
cosine_similarity

0.041746195

In [30]:
import numpy as np

def normalize(word_vec):
    norm=np.linalg.norm(word_vec)
    if norm == 0: 
        return word_vec
    return word_vec/norm

In [31]:
a=soma
b=soma2

In [32]:
a=normalize(a)

In [33]:
b=normalize(b)

In [34]:
from numpy import dot
from numpy.linalg import norm
cos_sim = dot(a, b)/(norm(a)*norm(b))
cos_sim

0.0417462